# Functions

## Imports & logging

In [1]:
import json
import logging

import geopandas as gpd
import numpy as np
import pandas as pd
import s3fs as s3
import xarray as xr

In [2]:
logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

## Datacube access & sampling

In [5]:
def get_datacube_instances(points):
    """Identifies the ITS_LIVE Zarr datacubes corresponding to sampling points.

    Parameters
    ----------
    points : geopandas.GeoDataFrame
        GeoDataFrame containing points along a glacier centreline.

    Returns
    -------
    datacubes : list of str
        List unique datacube URLs.
    points : geopandas.GeoDataFrame
        points with corresponding datacube URL appended.

    See also
    --------
    sample_datacubes
    """

    # Reads ITS_LIVE Zarr datacube catalog as GeoDataFrame
    s3fs = s3.S3FileSystem(anon=True)
    catalog_url = "s3://its-live-data/datacubes/catalog_v02.json"
    with s3fs.open(catalog_url, "r") as datacube_catalog:
        datacube_catalog = json.load(datacube_catalog)
        datacube_catalog = gpd.GeoDataFrame.from_features(datacube_catalog, crs=4326)

    # Intersects points with datacube outlines
    points = (
        points.to_crs(4326)
        .overlay(datacube_catalog[["geometry", "zarr_url"]], how="intersection")
        .set_index(["glacier", "cd"])
        .to_crs(3413)
    )

    # Amends URLs
    points["zarr_url"] = (
        points["zarr_url"].str.replace("http:", "s3:").str.replace(".s3.amazonaws.com", "")
    )

    # Creates list of unique datacubes (as their URLs)
    datacubes = points["zarr_url"].unique().tolist()
    logger.info(f"Number of unique datacubes: {len(datacubes)}")

    return datacubes, points

In [6]:
def sample_datacubes(datacubes, points, spatial_averaging=True):
    """Samples ITS_LIVE Zarr datacube at points.

    Parameters
    ----------
    datacubes : list of str
        List unique datacube URLs.
    points : geopandas.GeoDataFrame
        GeoDataFrame containing points along a glacier centreline.
    spatial_averaging : bool
        Flag to sample in a 3x3 window around each point [default=True].

    Returns
    -------
    datasets : list of xarray.Dataset
        Sampled datasets with the dimensions:
            - mid_date  ...
            - glacier   ...
            - cd        binned distance along centreline
            - k         int (0-8) each point in the 3x3 kernel

    See also
    --------
    get_datacube_instances
    """

    # Creates storage needed for spatial averaging, if necessary
    if spatial_averaging:
        points["windows"] = pd.Series(dtype=object)

    # Creates storage for sampled datasets
    datasets = []

    for dc_url in datacubes:
        # Gets index of points within current the datacube
        z_idx = points["zarr_url"] == dc_url

        with xr.open_dataset(dc_url, engine="zarr", storage_options={"anon": True}) as dc:
            if not spatial_averaging:
                # Gets point x and y coords
                point_x = points.loc[z_idx, "x"].to_xarray()
                point_y = points.loc[z_idx, "y"].to_xarray()
                mask = ~np.isnan(point_x)

                # Queries datacube at point
                dc = dc.sel(x=point_x, y=point_y, method="nearest").sortby("mid_date")

            else:
                # Creates additional points for 3x3 kernal about each index
                diffs_x = np.abs(points.loc[z_idx, "x"].values - dc["x"].values[:, np.newaxis])
                diffs_y = np.abs(points.loc[z_idx, "y"].values - dc["y"].values[:, np.newaxis])
                x_idx = diffs_x.argmin(axis=0)
                y_idx = diffs_y.argmin(axis=0)
                windows = []
                for x, y in list(zip(x_idx, y_idx)):
                    window = []
                    for y_ in [y + 1, y, y - 1]:
                        for x_ in [x - 1, x, x + 1]:
                            window.append((x_, y_))
                    windows.append(window)

                # Appends window indicies to points GeoDataFrame
                points.loc[z_idx, "windows"] = pd.Series(
                    windows, index=points.loc[z_idx].index
                )

                # Unpacks list of windows
                points = (
                    points.loc[z_idx, "windows"]
                    .apply(pd.Series)
                    .reset_index()
                    .melt(["glacier", "cd"])
                    .sort_values(by=["glacier", "cd"])
                    .rename(columns={"variable": "k", "value": "idx"})
                    .set_index(["glacier", "cd", "k"])
                )

                # Unpacks each window's tuple
                points = (
                    points["idx"].apply(pd.Series).rename(columns={0: "x_idx", 1: "y_idx"})
                )

                # Removes invalid indicies caused when points cross a datacube boundary
                invalid = (
                    (points["y_idx"] >= len(dc["y"]))
                    | (points["y_idx"] < 0)
                    | (points["x_idx"] >= len(dc["x"]))
                    | (points["x_idx"] < 0)
                )
                points = points[~invalid]

                # Gets point x and y coords
                point_x = points["x_idx"].to_xarray()
                point_y = points["y_idx"].to_xarray()
                mask = ~np.isnan(point_x)

                # Handles NaN values created when dealing with centrelines of differing length
                point_x = xr.where(mask, point_x, 0).astype(int)
                point_y = xr.where(mask, point_y, 0).astype(int)

                # Queries datacube at point
                dc = dc.isel(x=point_x, y=point_y).sortby("mid_date")

            # Masks NaN values
            dc["x"] = xr.where(mask, dc["x"], np.nan)
            dc["y"] = xr.where(mask, dc["y"], np.nan)
            dc["mask"] = mask

            logger.info(f"Datacube dimensions: {dc.dims}")
            datasets.append(dc.chunk(chunks={"cd": 15, "glacier": 1}))

    logger.info(f"Number of datacubes loaded: {len(datasets)}")

    return datasets

## Processing

### Error weighting & uncertainty

In [7]:
def error_weighted_average(values, error, resample_frequency="AS"):
    """Calculates error weighted average.

    Parameters
    ----------
    values : xarray.DataArray
        Values to average.
    error : xarray.DataArray
        Error associated with each value.
    resample_frequency : str
        Resampling frequency.

    Returns
    -------
    weighted_average : xarray.DataArray
        Error weighted average, resampled to the specified frequency.
    combined_uncertainty : xarray.DataArray
        Associated uncertainty.
    std : xarray.DataArray
        Associated standard deviations.

    See also
    --------
    get_annual_averages
    get_seasonal_averages
    get_time_series
    v_uncertainty
    """

    empty = np.zeros(values.shape)
    empty[:] = np.nan

    error_masked = xr.where(np.isnan(values), empty, error)
    weights = 1 / (error_masked**2)

    top = (weights * values).resample(mid_date=resample_frequency).sum()
    bottom = weights.resample(mid_date=resample_frequency).sum()
    weighted_average = top / bottom

    combined_uncertainty = np.sqrt(1 / (weights.resample(mid_date=resample_frequency).sum()))
    std = values.resample(mid_date=resample_frequency).std()

    return (
        weighted_average.rename("v"),
        combined_uncertainty.rename("v_error"),
        std.rename("std"),
    )

In [8]:
def v_uncertainty(vx, vy, vx_error=0, vy_error=0):
    """Calculates velocity and associated uncertainty from vx and vy components.

    Parameters
    ----------
    vx : xarray.DataArray
        vx velocity component.
    vy : xarray.DataArray
        vy velocity component.
    vx_error : xarray.DataArray
        Error associated with vx velocity component.
    vy_error : xarray.DataArray
        Error associated with vy velocity component.

    Returns
    -------
    v : xarray.DataArray
        Resultant velocity, calculated from vx and vy components.
    v_error : xarray.DataArray
        Associated uncertainty.

    Notes
    -----
    v = sqrt(vx^2 + vy^2)

    See also
    --------
    error_weighted_average
    get_annual_averages
    get_seasonal_averages
    get_time_series
    """

    def pow_err(q, x, dx, n):
        """Returns uncertainty in q (dq), where q = x^n and x has uncertainty dx
        dq / |q| = |n|* dx / |x|
        """
        return np.abs(q) * np.abs(n) * (dx / np.abs(x))

    vx2_error = pow_err(vx**2, vx, vx_error, 2)  # error in vx^2
    vy2_error = pow_err(vy**2, vy, vy_error, 2)  # error in vy^2

    sq_sum = vx**2 + vy**2
    sq_sum_error = np.hypot(vx2_error, vy2_error)  # error in vx^2 + vy^2

    v = np.sqrt(sq_sum)  # resultant velocity
    v_error = pow_err(v, sq_sum, sq_sum_error, 0.5)  # error in resultant velocity

    return v, v_error

### Annual averages

In [16]:
def get_annual_averages(datasets, date_dt_min, date_dt_max):
    """Calculates average annual velocities.

    Parameters
    ----------
    datasets : list of xarray.DataArray
        Sampled IT_LIVE data.
    date_dt_min : str
        pandas timedelta string specifying minimum date_dt.
    date_dt_max : str
        pandas timedelta string specifying minimum date_dt.

    Returns
    -------
    v_annual : pandas.DataFrame
        DataFrame of average annual velocities and their associated error.
    v_all : pandas.DataFrame
        DataFrame of spatially averaged velocities at *all* timesteps in the original xarray.Dataset (useful
        for plotting standard deviations with seaborn using .groupby(['glacier','cd','year]))

    Notes
    -----
    Filters sampled velocity data to only include that derived from image pairs with a
    separation time (date_dt) between date_dt_min and date_dt_max.

    See also
    --------
    error_weighted_average
    get_seasonal_averages
    get_time_series
    v_uncertainty
    """

    annual_v = []
    annual_v_error = []

    for ds in datasets:
        # Filter date_dt between date_dt_min and date_dt_max
        date_dt_idx = (ds["date_dt"] >= pd.Timedelta(date_dt_min)) & (
            ds["date_dt"] <= pd.Timedelta(date_dt_max)
        )

        if "k" not in list(ds.dims):
            # No spatial averaging
            v, v_error, _ = error_weighted_average(
                ds["v"][date_dt_idx, :, :], ds["v_error"][date_dt_idx, :, :]
            )
            annual_v.append(v)
            annual_v_error.append(v_error)

        else:
            # Calculates average velocity within each kernal from vx and vy components
            average_vx = ds["vx"][date_dt_idx.compute(), :, :, :].mean(dim="k", skipna=True)
            average_vx = ds["vy"][date_dt_idx.compute(), :, :, :].mean(dim="k", skipna=True)
            average_v, average_v_error = v_uncertainty(
                average_vx,
                average_vx,
                ds["vx_error"][date_dt_idx.compute()],
                ds["vy_error"][date_dt_idx.compute()],
            )

            v, v_error, _ = error_weighted_average(average_v, average_v_error)

            output_shape = dict(zip(v.dims, v.shape))
            valid_idx = np.repeat(
                (ds["mask"].mean(dim="k").values == 1)[np.newaxis, :, :],
                repeats=output_shape["mid_date"],
                axis=0,
            )

            v = xr.where(valid_idx, v, np.nan)
            v_error = xr.where(valid_idx, v_error, np.nan)
            annual_v.append(v)
            annual_v_error.append(v_error)

    # Merge annual datasets
    v = xr.merge(annual_v)["v"]
    v_error = xr.merge(annual_v_error)["v_error"]

    # Convert to DataFrame for better control over plotting
    v_error_annual = v_error.to_dataframe(
        name="v_error", dim_order=["glacier", "cd", "mid_date"]
    )
    v_annual = v.to_dataframe(name="v", dim_order=["glacier", "cd", "mid_date"])
    v_annual = v_annual.merge(v_error_annual, left_index=True, right_index=True)

    v_annual["year"] = v_annual.index.get_level_values(level=2).year
    v_annual["pcnt"] = (
        v_annual["v"]
        / v_annual.loc[
            v_annual.groupby(["glacier", "cd"])["year"].transform("idxmin"), "v"
        ].values
    ) * 100
    v_annual.reset_index(inplace=True)

    v_all = average_v.to_dataframe(name="v", dim_order=["glacier", "cd", "mid_date"])
    v_all["year"] = v_all.index.get_level_values(level=2).year
    v_all.reset_index(inplace=True)

    return v_annual, v_all

### Seasonal averages

In [ ]:
def getSeasonalAverages(dss, startD, stopD, ddt_max="30D", rsmpFreq="Q-FEB"):
    """
    get seasonally averaged velocities
    input:  dss - list of xr.Datasets dims = [mid_date, glacier, cd, (k)]
            startD, stopD - pandas datetimes for filtering on mid_date
            ddt_max - pandas dateoffset string for filtering date_dt - default 30 days
            rsmpFreq - frequnecy to resample to, default = Q-FEB
    returns: sv_df : a pandas data frame with columns:
                glacier, cd, mid_date (with frequency == rsmpFreq), v, season, year
    """

    seasonalVs = []
    # seasonalErrs = []
    # seasonalStds = []
    for ds in dss:
        # date range
        d_idx = ds["mid_date"].to_pandas().between(startD, stopD)
        # date_dt filter
        ddt_idx = ds["date_dt"] <= pd.Timedelta(ddt_max)
        # combined mid_date filter
        mid_date_idx = ddt_idx & d_idx

        if "k" in list(ds.dims):  # spatialAveraging: if spatialAveraging
            # compute average velocity in each kernal, from vx,vy components
            avg_vx = ds["vx"][mid_date_idx, :, :, :].mean(dim="k", skipna=True)
            avg_vy = ds["vy"][mid_date_idx, :, :, :].mean(dim="k", skipna=True)
            avg_v, avg_v_err = v_uncert(
                avg_vx, avg_vy, ds["vx_error"][mid_date_idx], ds["vy_error"][mid_date_idx]
            )
            # # or just do ds['v'].mean(dim = 'k')...and use the supplied 'v_error'
            seasonalV, _, _ = errWgtAvg(avg_v, avg_v_err, freq=rsmpFreq)

            output_shape = dict(zip(seasonalV.dims, seasonalV.shape))
            valid_idx = np.repeat(
                (ds["mask"].mean(dim="k").values == 1)[np.newaxis, :, :],
                repeats=output_shape["mid_date"],
                axis=0,
            )

            seasonalV = xr.where(valid_idx, seasonalV, np.nan)
            # seasonalErr = xr.where(valid_idx, seasonalErr, np.nan)
            # seasonalStd = xr.where(valid_idx, seasonalStd, np.nan)

            seasonalVs.append(seasonalV)
            # seasonalErrs.append(seasonalErr)
            # seasonalStds.append(seasonalStd)

        else:
            avg_v = ds["v"][mid_date_idx, :, :]
            seasonalV, _, _ = errWgtAvg(
                ds["v"][mid_date_idx, :, :], ds["v_error"][mid_date_idx, :, :]
            )
            # seasonalVs.append(seasonalV)
            # seasonalErrs.append(seasonalErr)
            seasonalVs.append(seasonalV)

    Sv = xr.merge(seasonalVs)["v"]

    ## seasonal trends along centreline constructed by first calculating seasonal averages (1 per year)
    # then computing trend
    ssn = {1: "DJF", 2: "MAM", 3: "JJA", 4: "SON"}
    sv_df = Sv.to_dataframe(dim_order=["glacier", "cd", "mid_date"])
    sv_df.reset_index(inplace=True)
    sv_df["season"] = (sv_df["mid_date"].dt.month % 12 // 3 + 1).map(ssn)
    sv_df["year"] = sv_df["mid_date"].dt.year

    return sv_df

### Time series

In [ ]:
def getTimeseries(dss, startD, stopD, distances, ddt_max="30D"):
    """
    for formatting timeseries at a point(s) along centreline, and resampling them to regular frequnecies
    inputs: dss - list of xarray datasets, dims(mid_date, glacier, cd, (k))
            startD, stopD - pandas datetimes for filtering - mid_date
            distances - list of distances along centreline (cd) to include
            ddt_max - pandas date offset string (e.g. '30D'. for filtering date_dt. only include date_dts <= ddt_max

    returns: ts_df:
        pandas dataframe: velocities that meet date, date_dt and cd filter requirements (spatially averaged if `k` is present in list of dims)
        w/  rolling 28 d average
    """
    filtered = []
    for ds in dss:
        # filter on startD, stopD and ddt_max
        d_idx = ds["mid_date"].to_pandas().between(startD, stopD)
        ddt_idx = ds["date_dt"] <= pd.Timedelta(ddt_max)
        mid_date_idx = ddt_idx & d_idx

        ## filter on distances
        cd_idx = ds["cd"].isin(distances)

        if "k" in list(ds.dims):  # then spatial averaging
            # average spatially in kernal.
            avg_vx = ds["vx"][mid_date_idx, :, cd_idx, :].mean(dim="k", skipna=True)
            avg_vy = ds["vy"][mid_date_idx, :, cd_idx, :].mean(dim="k", skipna=True)
            avg_v, avg_v_err = v_uncert(
                avg_vx, avg_vy, ds["vx_error"][mid_date_idx], ds["vy_error"][mid_date_idx]
            )

        else:
            avg_v, avg_v_err = v_uncert(
                ds["vx"][mid_date_idx, :, cd_idx],
                ds["vy"][mid_date_idx, :, cd_idx],
                ds["vx_error"][mid_date_idx],
                ds["vy_error"][mid_date_idx],
            )

        # merge v, v_err and meta data
        filtered.append(
            xr.merge(
                [
                    avg_v.rename("v"),
                    avg_v_err.rename("err"),
                    ds[
                        [
                            "acquisition_date_img1",
                            "acquisition_date_img2",
                            "date_dt",
                            "satellite_img1",
                            "mission_img1",
                        ]
                    ].sel(mid_date=mid_date_idx),
                ]
            )
        )

    # merge list of filterd dataframes and convert to dask_df
    ds_out = xr.merge(filtered)
    ts_df = ds_out.to_dataframe()

    # # drop nan rows
    ts_df = ts_df[~ts_df["v"].isna()]

    # rolling median
    ts_df = ts_df.merge(
        ts_df.reset_index(level=[1, 2])
        .groupby(["glacier", "cd"])["v"]
        .rolling("28D")
        .median()
        .rename("smoothed_v")
        .reset_index(),
        left_on=["mid_date", "glacier", "cd"],
        right_on=["mid_date", "glacier", "cd"],
    )

    # calculate intermediate steps for error-weighted averages
    ts_df["v_over_sigma2"] = ts_df["v"] / (ts_df["err"] ** 2)
    ts_df["one_over_sigma2"] = 1 / (ts_df["err"] ** 2)

    return ts_df

# Analysis

## Imports & set-up

In [10]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from dask.distributed import Client
from scipy.stats import linregress

In [11]:
# Defines plot settings
plt.rcParams["axes.labelsize"] = 14
plt.rcParams["figure.dpi"] = 300
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 11
plt.rcParams["mathtext.fontset"] = "stix"
plt.rcParams["xtick.labelsize"] = 11
plt.rcParams["ytick.labelsize"] = 11

In [12]:
dask_client = Client()
logger.info(f"Dask dashboard: {dask_client.dashboard_link}")

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.diskutils:Found stale lock file and directory 'C:\\Users\\olive\\AppData\\Local\\Temp\\dask-scratch-space\\scheduler-b6p2inuk', purging
INFO:distributed.diskutils:Found stale lock file and directory 'C:\\Users\\olive\\AppData\\Local\\Temp\\dask-scratch-space\\scheduler-kwgaa1eb', purging
INFO:distributed.diskutils:Found stale lock file and directory 'C:\\Users\\olive\\AppData\\Local\\Temp\\dask-scratch-space\\scheduler-t7zmoy4h', purging
INFO:distributed.diskutils:Found stale lock file and directory 'C:\\Users\\olive\\AppData\\Local\\Temp\\dask-scratch-space\\worker-8vi7rv52', purging
INFO:distributed.diskutils:Found stale lock file and directory 'C:\\Users\\olive\\AppData\\Local\\Temp\\dask-scratch-space\\worker-hm3x32i8', purging
INFO:distributed.diskutils:Found stale lock file an

## Read points

In [13]:
# Reads sampling points
points_path = "C:\\Users\\olive\\Projects\\diss\\src\\holt_isortuarsuup-sermia_10-5281-zendono-7824987\\tlohde-isortuarsuupSermia-396b33c\\data\\samplePoints_feather"
points = gpd.read_feather(points_path)

selection = ["IsortuarsuupSermia", "KangaasarsuupSermia"]

## Sample datacubes

In [14]:
# EXPECTED RUNTIME: ~3 minutes

# Identifies and sample ITS_LIVE Zarr datacubes that intersect with sampling points
datacubes, points = get_datacube_instances(points)
datasets = sample_datacubes(datacubes, points, spatial_averaging=True)

INFO:__main__:Number of unique datacubes: 1
C:\Users\olive\AppData\Local\Temp\ipykernel_21332\3837882313.py:70: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  .apply(pd.Series)
C:\Users\olive\AppData\Local\Temp\ipykernel_21332\3837882313.py:80: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  points["idx"].apply(pd.Series).rename(columns={0: "x_idx", 1: "y_idx"})
INFO:__main__:Datacube dimensions: Frozen({'mid_date': 42808, 'glacier': 2, 'cd': 69, 'k': 9})
INFO:__main__:Number of datacubes loaded: 1


## Annual averages

In [17]:
# EXPECTED RUNTIME: ~2 minutes

# Calculates annual averages
date_dt_min = "300D"  # 300 days
date_dt_max = "430D"  # 430 days
v_annual, v_all = get_annual_averages(datasets, date_dt_min, date_dt_max)

# Calculates annual trends
annual_trend = (
    v_annual.groupby(["glacier", "cd"])
    .apply(lambda q: pd.Series(linregress(x=q["year"], y=q["v"])))
    .reset_index()
    .rename(columns={0: "slope", 1: "intercept", 2: "rvalue", 3: "pvalue", 4: "stderr"})
)

# Converts from metres to kilometres
v_all["cd_km"] = v_all["cd"] / 1000
v_annual["cd_km"] = v_annual["cd"] / 1000
annual_trend["cd_km"] = annual_trend["cd"] / 1000

C:\Users\olive\AppData\Roaming\Python\Python311\site-packages\distributed\client.py:3169: UserWarning: Sending large graph of size 1.20 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
C:\Users\olive\AppData\Roaming\Python\Python311\site-packages\distributed\client.py:3169: UserWarning: Sending large graph of size 1.20 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
C:\Users\olive\AppData\Roaming\Python\Python311\site-packages\distributed\client.py:3169: UserWarning: Sending large graph of size 1.19 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
# annual velocities, percentage, and trends
fig, axs = plt.subplots(ncols=2, nrows=3, figsize=(10, 8), sharex=True, sharey="row")

for i, g in enumerate(selection):
    # annual velocity with standard deviation as measure of spread
    idx = (v_all["glacier"] == g) & (v_all["year"] < 2022)
    if i == 1:
        lgnd = "full"
    else:
        lgnd = False
    sns.lineplot(
        data=v_all.loc[idx, :],
        x="cd_km",
        y="v",
        errorbar="sd",  # with standard deviation as measure of spread
        hue="year",
        style="year",
        ax=axs[0, i],
        legend=lgnd,
    )
    axs[1, i].set_ylim(top=251)
    axs[0, i].set_title(g)

    # percentage change
    idx = (v_annual["glacier"] == g) & (v_annual["year"] < 2022)
    sns.lineplot(
        data=df.loc[idx],
        x="cd_km",
        y="pcnt",
        hue="year",
        style="year",
        ax=axs[1, i],
        legend=False,
    )
    axs[1, i].set_ylim(75, 280)

    # acceleration - significant only
    idx = (annual_trend["glacier"] == g) & (annual_trend["pvalue"] <= 0.05)
    axs[2, i].errorbar(
        x=annual_trend.loc[idx, "cd_km"],
        y=annual_trend.loc[idx, "slope"],
        yerr=annual_trend.loc[idx, "stderr"],
        ecolor="grey",
        fmt=".",
        color="k",
    )
    axs[2, i].set_ylim(-2, 21)

# label axes
labs = "abcdefghi"
for i, ax in enumerate(axs.flat):
    ax.set_xlim(-0.150, 16.150)
    ax.annotate(f"({labs[i]})", xy=(0.9, 0.87), xycoords="axes fraction", fontsize=18)
    sns.despine(ax=ax, trim=True)

###### add inset to KS % velocity plot
axins = axs[1, 1].inset_axes([0.12, 0.46, 0.75, 0.48])
idx = (v_annual["glacier"] == "KangaasarsuupSermia") & (v_annual["year"] < 2022)
sns.lineplot(
    data=v_annual.loc[idx],
    x="cd_km",
    y="pcnt",
    hue="year",
    style="year",
    ax=axins,
    legend=False,
)

# sub region of the original image
x1, x2, y1, y2 = 2.5, 8, 95, 140
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels([])
axins.set_yticks(axins.get_yticks()[1:3])
axins.set_ylabel(None)
axins.set_xlabel(None)
axins.set_yticklabels(
    [int(q) for q in axins.get_yticks()], rotation=90, ha="center", va="center"
)
axs[1, 1].indicate_inset_zoom(axins, edgecolor="black")

# label axes
axs[0, 0].set_ylabel("Velocity (m yr$^{-1}$)")
axs[1, 0].set_ylabel("Velocity\nchange (%)")
axs[2, 0].set_ylabel("Velocity\ntrend (m yr$^{-2}$)")
axs[2, 0].set_xlabel("Distance from terminus (km)")
axs[2, 1].set_xlabel("Distance form terminus (km)")

# tidy up legend
sns.move_legend(
    axs[0, 1],
    loc="upper left",
    bbox_to_anchor=(0, 1.05),
    ncol=3,
    fontsize=12,
    title="Year",
    labelspacing=0.1,
    columnspacing=0.5,
    frameon=False,
)

leg = axs[0, 1].get_legend()
for legobj in leg.legend_handles:
    legobj.set_linewidth(2)
plt.subplots_adjust(hspace=0.08, wspace=0.05)

# fig.savefig("../figures/f02.png", bbox_inches="tight")

print(
    "Figure 2: Annual average ice velocity (2013–2021) profiles along centrelines shown in Figure 1 at\n\
    (a) Isortuarsuup Sermia, and (b) Kangaasarsuup Sermia. (c) and (d) show percentage change relative to 2013. \n\
        (e) and (f) show linear trends where regression slope coefficients are significant at p ≤ .05; \n\
            error bars denote standard error of the estimate."
)

## Time series

In [ ]:
# takes ~ 40 seconds
startD = pd.to_datetime("2016-01-01")
stopD = pd.to_datetime("2022-01-01")
# distances = np.arange(500,10500,500)
distances = [1000, 10000]
ddt_max = "30D"

# get timeseries timeseries at each point/glacier
ts_df = h.getTimeseries(dss, startD, stopD, distances, ddt_max)
ts_df["day_sep"] = ts_df["date_dt"] / pd.Timedelta("1D")

In [ ]:
### plotting with simple median filter
fg = sns.relplot(
    data=(
        ts_df.set_index("mid_date")
        .groupby(["glacier", "cd"])["v"]
        .rolling("28D", min_periods=10, center=True)
        .median()
        .rename("smoothed_v")
        .reset_index()
    ),
    x="mid_date",
    y="smoothed_v",
    hue="glacier",
    palette="colorblind",
    style="glacier",
    linewidth=2,
    row="cd",
    kind="line",
    aspect=1.8,
)

for d in distances:
    tsdf_idx = ts_df["cd"] == d
    sns.scatterplot(
        data=ts_df.loc[tsdf_idx],
        x="mid_date",
        y="v",
        hue="glacier",
        palette="colorblind",
        style="glacier",
        alpha=0.8,
        s=2,
        ax=fg.axes_dict[d],
        legend=False,
    )

sns.move_legend(fg, loc="upper right", bbox_to_anchor=(0.7, 0.5))  # (0.75, 0.53)

fg.set_titles("")

labs = "abcdefg"
for i, ax in enumerate(fg.axes.flatten()):
    ax.annotate(f"({labs[i]})", xy=(0.05, 0.88), xycoords="axes fraction", fontsize=18)
    ax.annotate(
        f"{str(int(distances[i]/1000))} km from terminus",
        xy=(0.1, 0.88),
        xycoords="axes fraction",
        fontsize=12,
    )
    # ax.set_title(f'Distance from terminus: {str(int(distances[i]/1000))} km', y=0.82, loc='left')

fg.set(ylim=(-10, 340))
fg.set_xticklabels([2016, 2017, 2018, 2019, 2020, 2021, 2022])
fg.set(ylabel=("Velocity (m yr$^{-1}$)"), xlabel=("Year"))
fg.despine(trim=True)

plt.subplots_adjust(hspace=0.01)

# fg.fig.savefig('../figures/f03.png',
#                bbox_inches='tight')

print(
    "Figure 3: Time series of ice surface velocity at (a) 1 km and (b) 10 km from the terminus at\n\
      the lake-terminating Isortuarsuup Sermia (blue) and\n\
            the land-terminating Kangaasarsuup Sermia (orange, dashed). \n\
      These velocities are computed from image-pairs separated by ≤ 30 days.\n\
            Lines show the rolling 28 day median."
)

## Seasonal averages

In [ ]:
# takes ~40 seconds
##### compute seasonal averages and their trends along glacier centrelines
startD = pd.to_datetime("2016-01-01")
stopD = pd.to_datetime("2022-01-01")
ddt_max = "30D"
rsmpFreq = "Q-FEB"

sv_df = h.getSeasonalAverages(dss, startD, stopD, ddt_max, rsmpFreq)

## compute seasonal trends
ssn_trend = (
    sv_df.groupby(["glacier", "season", "cd"])
    .apply(lambda q: pd.Series(linregress(x=q["year"], y=q["v"])))
    .reset_index()
    .rename(columns={0: "slope", 1: "intercept", 2: "rvalue", 3: "pvalue", 4: "stderr"})
)

ssn_trend["significant\np<0.05"] = ssn_trend["pvalue"] <= 0.05
ssn_trend["Distance (km)"] = ssn_trend["cd"] / 1000

In [ ]:
# color palette. from PuOr from colorbrewer. should be BW and colorblind safe
rgbs = [(94, 60, 153), (253, 184, 99), (230, 97, 1), (178, 171, 210)]
rgbs = [tuple(i / 255 for i in a) for a in rgbs]
season_colors = dict(zip(["DJF", "MAM", "JJA", "SON"], rgbs))

## plot seasonal trends - color and style by season. size by significance
fg = sns.relplot(
    data=ssn_trend,
    x="Distance (km)",
    y="slope",
    hue="season",
    hue_order=["DJF", "MAM", "JJA", "SON"],
    palette=rgbs,
    style="season",
    style_order=["DJF", "MAM", "JJA", "SON"],
    markers=["o", "d", "^", "P"],
    size="significant\np<0.05",
    size_order=[True, False],
    col="glacier",
    legend=False,
)

fg.map(plt.axhline, y=0, color="grey", zorder=0.5)
fg.set(xlim=(-0.5, 16), ylim=(-15.5, 26.5), yticks=np.arange(-15, 30, 5))
fg.set_ylabels("Velocity trend (m yr$^{-2}$)")
fg.despine(trim=True)

# label axes
labs = "abcdef"
for i, ax in enumerate(fg.axes.flat):
    ax.annotate(f"({labs[i]})", xy=(0.05, 0.94), xycoords="axes fraction", fontsize=18)
    t = ax.get_title()
    ax.set_title(t.split("glacier = ")[1], y=0.92)
    ax.set_xlabel("Distance from terminus (km)")

# error bars
for g in selection:
    for S in ["DJF", "MAM", "JJA", "SON"]:
        idx = (
            (ssn_trend["glacier"] == g)
            & (ssn_trend["season"] == S)
            & (ssn_trend["significant\np<0.05"] == True)
        )
        fg.axes_dict[g].errorbar(
            x=ssn_trend.loc[idx, "Distance (km)"],
            y=ssn_trend.loc[idx, "slope"],
            yerr=ssn_trend.loc[idx, "stderr"],
            fmt="None",
            zorder=0.5,
            alpha=0.3,
            ecolor=season_colors[S],
        )

# ridiculous routine. make and plot some dummy data in order to construct two separate legends
# because just using the `ncols` paramter splits the seasons over two columns, which is ugly
dummy = pd.DataFrame(
    {
        "a": [-5, -5, -5, -5],
        "b": [0, 0, 0, 0],
        "season": ["Winter (DJF)", "Spring (MAM)", "Summer (JJA)", "Autumn (SON)"],
    }
)
q = sns.scatterplot(
    data=dummy,
    x="a",
    y="b",
    hue="season",
    hue_order=["Winter (DJF)", "Spring (MAM)", "Summer (JJA)", "Autumn (SON)"],
    palette=rgbs,
    style="season",
    style_order=["Winter (DJF)", "Spring (MAM)", "Summer (JJA)", "Autumn (SON)"],
    markers=["o", "d", "^", "P"],
    ax=fg.axes[0][0],
    legend=True,
)
sns.move_legend(q, loc="upper left", bbox_to_anchor=(1.05, 0.94), frameon=False)

dummy = pd.DataFrame({"a": [-5, -5], "b": [0, 0], "significant\np<0.05": [True, False]})
r = sns.scatterplot(
    data=dummy,
    x="a",
    y="b",  # palette=[rgbs[0],rbgs[2]],
    size="significant\np<0.05",
    size_order=[True, False],
    ax=fg.axes[0][1],
)

sns.move_legend(r, loc="upper left", bbox_to_anchor=(0.4, 0.94), frameon=False)
fg.axes[0][0].set_zorder(2)

# fg.fig.savefig('../figures/f04.png', bbox_inches='tight')

print(
    "Figure 4: Seasonal velocity trends (2016–2021) along glacier centrelines at (A) Isortuarsuup Sermia and (B) Kangaasarsuup Sermia for\n\
    winter (DJF, purple circles), spring (MAM, gold diamonds), summer (JJA, orange triangles) and\n\
        autumn (SON, grey crosses). Seasonal trends derived from velocity fields were “date_dt” ≤ 30 days. \n\
            Trends shown are linear fits. Significant (p ≤ .05) trends shown by larger symbols, otherwise smaller symbols;\n\
                error bars denote standard error of the estimate."
)